<a href="https://colab.research.google.com/github/srnarasim/DataProcessingComparison/blob/main/scenario1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install polars duckdb pyarrow

In [3]:
# Generate realistic sample data
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

def generate_sample_data(n_rows=500_000):
    """Generate realistic e-commerce transaction data"""
    np.random.seed(42)  # Reproducible results

    # Realistic data that shows meaningful differences between tools
    customers = [f"CUST_{i:06d}" for i in range(1, 10001)]
    products = [f"PROD_{i:04d}" for i in range(1, 1001)]

    data = {
        'customer_id': np.random.choice(customers, n_rows),
        'product_id': np.random.choice(products, n_rows),
        'order_total': np.random.lognormal(3, 1, n_rows).round(2),
        'order_date': [
            datetime(2024, 1, 1) + timedelta(days=np.random.randint(0, 365))
            for _ in range(n_rows)
        ]
    }

    return pd.DataFrame(data)

# This gives readers realistic data to experiment with
sample_data = generate_sample_data()
print(f"Generated {len(sample_data):,} realistic transactions")
sample_data.head()

Generated 500,000 realistic transactions


,customer_id,product_id,order_total,order_date
0,CUST_007271,PROD_0138,41.54,2024-10-27
1,CUST_000861,PROD_0353,23.62,2024-07-07
2,CUST_005391,PROD_0544,26.22,2024-01-31
3,CUST_005192,PROD_0498,3.78,2024-03-29
4,CUST_005735,PROD_0838,7.18,2024-07-29
